In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import tensorflow as tf
#import tensorflow_datasets as tfds
#from sklearn.model_selection import train_test_split

import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt 
from matplotlib import image
import random as rd
import math as m
from PIL import Image

import requests as req
import npzviewer
import wget
from bs4 import BeautifulSoup
import csv

import os
print(os.getcwd())

#from google.cloud import storage
#client = storage.Client()
import copy
#pip install --user google-cloud-storage

print(tf.__version__)


# In[2]:


##########


# In[2]:


def download_data(categories,number_of_samples):

    number_of_categories = len(categories)
    data = np.array([], dtype=np.int64).reshape(0,784)
    for i in range(number_of_categories):
        filename = cats[i][0] + '.npy'
        filename = filename.replace(" ","%20")
        
        url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'+filename
        
        r = req.get(url, allow_redirects=True)
        #filename = os.path.basename(url)
        #filename = filename.replace("%20","")
        open(filename, 'wb').write(r.content)
        
        data = np.vstack([data,load_data(filename,number_of_samples)])
        
        os.remove(filename)
        print(i+1,'/',number_of_categories,' ',filename.replace("%20",""))
    return data

def load_data(name,n):
    filename = name
    label = name
    data = np.load(filename)
    #data = np.ndarray.reshape(data,len(data),28,28)
    return data[0:n,:]

#randomize data and labels
def randomize_data(data,labels,seed):
    np.random.seed(seed)
    np.random.shuffle(data)
    np.random.seed(seed)
    np.random.shuffle(cat_id)


# In[3]:


#load categories
cats = open("categories.txt",'r')
reader = csv.reader(cats)
cats = [row for row in reader]
cats = cats[1:10]


number_of_categories = len(cats)
number_of_samples = 1000

cat_id = np.repeat(range(number_of_categories),number_of_samples)+1

#load data from google storage
#d = download_data(cats[0:number_of_categories],number_of_samples)
#np.savetxt('data_10_1000.csv', d, delimiter=',')

#load data from file
d = np.loadtxt('data_9_1000.csv', delimiter=',')
data = np.reshape(d,(len(d),28,28))

#reshape data into 28x28
#data = d.copy()
#data = np.reshape(data,(len(d),28,28))


# In[4]:


#randomize data and cat_id
randomize_data(data,cat_id,67)


# In[5]:


#TEST Show
i=rd.randint(1,number_of_categories*number_of_samples)
plt.imshow(data[i])
print(cats[cat_id[i]-1],i)


# In[8]:


#split data
training = 0.8
test = 1.-training

x_train = data[0:m.floor(training*len(data))]/ 255.0
y_train = cat_id[0:m.floor(training*len(cat_id))]

x_test = data[m.ceil(training*len(data)):len(data)]/ 255.0
y_test = cat_id[m.ceil(training*len(cat_id)):len(cat_id)]

print('Train-Set: ','Samples',np.shape(x_train)[0])
print('Test-Set: ','Samples',np.shape(x_test)[0])


# In[9]:


#TEST Show
i=rd.randint(1,number_of_categories*number_of_samples)
plt.imshow(x_train[i])
print(cats[cat_id[i]-1],i)


# In[12]:


x_train.shape


# In[13]:


model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(number_of_categories),
  tf.keras.layers.Softmax()
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


# In[ ]:


# Compile and Fit in Batches
x_train_len = len(x_train)
y_train_len = len(y_train)
#x_test_len = len(x_test)
#y_test_len = len(y_test)

#batches = int(number_of_categories/5)
#end = -1
#for i in range(batches):
#    
#    start = int(end+1)
#    end = int((i+1)*1/batches*x_train_len)
#    
#    x_train = x_train[start:end,:,:]
#    y_train = y_train[start:end]
#    #x_test = x_test[start:end,:]
#    #y_test = y_test[start:end]
#
#    # Run with new data
#    model.compile(optimizer='adam',
#                  loss=loss_fn,
#                  metrics=['accuracy'])
#    
#    model.fit(x_train, y_train, epochs=5)


# In[ ]:


model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)


model.evaluate(x_test,  y_test, verbose=2);




/Users/babi/Documents/GitHub/Quickdraw1/training_quickdraw
2.4.0
['angel'] 7729
Train-Set:  Samples 7200
Test-Set:  Samples 1800
['alarm clock'] 1025
Epoch 1/5
